In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit

spark = SparkSession.builder.getOrCreate()

import os
import glob

In [2]:
# Simple pattern to Install custom packages from Juypter.
username = 'mhk9c'
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install demoji
!{sys.executable} -m pip install tldextract

sys.path.append(f'/home/{username}/.local/lib/python3.7/site-packages/')

# Then you can import them.
import demoji 
demoji.download_codes()

import tldextract

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


/opt/conda/lib/python3.7/site-packages/ipykernel/__main__.py:12: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.


### I propose that we keep all of our data, downloaded or derived, in the common folder

In [3]:
data_path = "/project/ds5559/team1_sp22/data/"

In [5]:
def save_df(_df, name):
    # Check whether the specified path exists or not
    full_path = f'{data_path}{name}'
    print(full_path)  
    if not os.path.exists(full_path):  
        # Create a new directory because it does not exist 
        os.makedirs(full_path)
        print("The new directory is created!")
    
    _df.write.format("parquet").mode("overwrite").save(f"{full_path}")
    os.system(f'chmod -R 777 {full_path}')
    print(f'Saved as: {full_path}')

def load_data(name):       
    full_path = f'{data_path}/{name}'
    _df = spark.read.parquet(full_path)        
    print(f'Done loading from {full_path}.')
    return _df
    
def create_df_from_csv(name):
    first = True
    for file in glob.glob(f'{data_path}/{name}/*.csv'):            
        print(file)
        if(first):
            _df = spark.read.csv(file, header=True, inferSchema=True, mode="DROPMALFORMED")                
            _df = _df.withColumn("source_file",lit(file))
        else:
            new_df = spark.read.csv(file, header=True, inferSchema=True, mode="DROPMALFORMED")
            new_df = new_df.withColumn("source_file",lit(file))                
            _df = _df.union(new_df)                        
        first = False        
    return _df
    


        
        
df = create_df_from_csv("russian-troll-tweets-master")
# df = load_data("russian-troll-tweets")
# save_df(df, "russian-troll-tweets")

total_tweets = df.count()
print(f'There are {total_tweets} tweets in this dataset')

/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_13.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_9.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_3.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_12.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_5.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_2.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_6.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_11.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_4.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_7.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_tweets_10.csv
/project/ds5559/team1_sp22/data//russian-troll-tweets-master/IRAhandle_t

In [6]:
save_df(df, "russian-troll-tweets")

/project/ds5559/team1_sp22/data/russian-troll-tweets
Saved as: /project/ds5559/team1_sp22/data/russian-troll-tweets


In [ ]:
# df.printSchema()

In [ ]:
df.createOrReplaceTempView("tweets")

In [ ]:
sqlDF = spark.sql("SELECT * FROM tweets where language = 'English' ")
english_tweets = sqlDF.count()
print(f'There are {english_tweets:,} english tweets in this dataset. They account for {english_tweets/total_tweets:%} of the dataset.')

### Add some additional columns to the data.

In [ ]:
import pyspark.sql.functions as func
from pyspark.sql.types import StringType, ArrayType
import re
b = re.compile(r"@[a-zA-Z0-9]+")

def convert_emojii(string):    
    return demoji.replace_with_desc(string, ":")
convert_emojii_UDF = func.udf(lambda z:convert_emojii(z),StringType())   
# test = convert_emojii("🐝🐝🐝")   
# print(test)


def extract_domain_information(url):
    if(url):
        ext = tldextract.extract(url)
        return ext.registered_domain
    else:
        return ""
extract_domain_information_UDF = func.udf(lambda z:extract_domain_information(z),StringType())   
# test = extract_domain_information("https://rivanna-portal.hpc.virginia.edu/node/udc-ba27-18/55477/lab?")
# print(test)


def extract_handles(content):     
    if(content is not None):        
        result = re.findall(b, content) 
        return result
    else:
        return []
extract_handles_UDF = func.udf(lambda z:extract_handles(z),ArrayType(StringType(), True))   
# test = extract_handles("Hi @MichelleObama , remember when you praised Harvey Weinstein as 'a wonderful human being, a good friend and a powerhouse.")
# print(test)


sqlDF = sqlDF.withColumn("curated_content", convert_emojii_UDF(col("content"))) \
                .withColumn("tco1_step1_domain", extract_domain_information_UDF(col("tco1_step1"))) \
                .withColumn("tco2_step1_domain", extract_domain_information_UDF(col("tco2_step1"))) \
                .withColumn("tco3_step1_domain", extract_domain_information_UDF(col("tco3_step1"))) \
                .withColumn("handles", extract_handles_UDF(col("content")))


sqlDF.select(["handles"]).show(100, False)

In [ ]:
sqlDF.createOrReplaceTempView("english_tweets")

In [ ]:
_sqlDF = spark.sql("SELECT tco1_step1, tco2_step1, tco3_step1  FROM english_tweets LIMIT 10")
_sqlDF.show(10, False)

In [ ]:
sqlDF = spark.sql("SELECT content,source_file FROM english_tweets LIMIT 100")